In [10]:
import duckdb
import pandas as pd
pd.set_option('display.max_columns', None)

import urllib.request
import os

conn = duckdb.connect('data/duck.db')

def download_url_to_file(url, filepath):
    if os.path.exists(filepath):
        print(f"File already exists at {filepath}")
        return
    response = urllib.request.urlopen(url)
    with open(filepath, 'wb') as file:
        file.write(response.read())
    print(f"File saved successfully at {filepath}")

download_url_to_file('https://nces.ed.gov/ipeds/datacenter/data/HD2023_Dict.zip', 'data/HD2023_Dict.zip')
download_url_to_file('https://nces.ed.gov/ipeds/datacenter/data/IC2023_Dict.zip', 'data/IC2023_Dict.zip')
download_url_to_file('https://nces.ed.gov/ipeds/datacenter/data/IC2023_AY_Dict.zip', 'data/IC2023_AY_Dict.zip')
download_url_to_file('https://nces.ed.gov/ipeds/datacenter/data/HD2023.zip', 'data/HD2023.zip')
download_url_to_file('https://nces.ed.gov/ipeds/datacenter/data/IC2023.zip', 'data/IC2023.zip')
download_url_to_file('https://nces.ed.gov/ipeds/datacenter/data/IC2023_AY.zip', 'data/IC2023_AY.zip')

print('All Files Downloaded Successfully')

File already exists at data/HD2023_Dict.zip
File already exists at data/IC2023_Dict.zip
File already exists at data/IC2023_AY_Dict.zip
File already exists at data/HD2023.zip
File already exists at data/IC2023.zip
File already exists at data/IC2023_AY.zip
All Files Downloaded Successfully


In [11]:
import zipfile
import os

# Unzip all zip files in the 'data' directory
data_dir = 'data'
for item in os.listdir(data_dir):
    if item.endswith('.zip'):
        file_path = os.path.join(data_dir, item)
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(data_dir)
        print(f"Unzipped {item} successfully")

Unzipped HD2023_Dict.zip successfully
Unzipped IC2023_AY_Dict.zip successfully
Unzipped HD2023.zip successfully
Unzipped IC2023_AY.zip successfully
Unzipped IC2023.zip successfully
Unzipped IC2023_Dict.zip successfully


In [12]:
import zipfile
import os
import subprocess
import glob

# Unzip all zip files in the 'data' directory
data_dir = 'data'
for item in glob.glob(os.path.join(data_dir, '*.csv')):
    if '-utf8' in item:
        print(f"Skipping {item} as it is already converted to UTF-8")
        continue
    name_only = os.path.basename(item).split('.')[0]
    utf_name = name_only + '-utf8.csv'
    script = f"iconv -f latin1 -t UTF-8 {item} > {os.path.join(data_dir, utf_name)}"
    print(f"Running: {script}")
    subprocess.run(args=['sh', '-c', script])
    print(f"Converted {item} to {utf_name} successfully")

Skipping data/hd2023-utf8.csv as it is already converted to UTF-8
Running: iconv -f latin1 -t UTF-8 data/ic2023_ay.csv > data/ic2023_ay-utf8.csv
Converted data/ic2023_ay.csv to ic2023_ay-utf8.csv successfully
Running: iconv -f latin1 -t UTF-8 data/hd2023.csv > data/hd2023-utf8.csv
Converted data/hd2023.csv to hd2023-utf8.csv successfully
Skipping data/ic2023_ay-utf8.csv as it is already converted to UTF-8
Skipping data/ic2023-utf8.csv as it is already converted to UTF-8
Running: iconv -f latin1 -t UTF-8 data/ic2023.csv > data/ic2023-utf8.csv
Converted data/ic2023.csv to ic2023-utf8.csv successfully


In [13]:
def parse_domain(url):
    if not url:
        return None
    parsed_url = urllib.parse.urlparse(url)
    domain = parsed_url.netloc.split(':')[0]  # Get the host part and remove port if present
    return domain


schools = pd.read_csv('data/hd2023.csv', encoding='latin1')
schools['domain'] = schools['WEBADDR'].apply(parse_domain)
schools

,UNITID,INSTNM,IALIAS,ADDR,CITY,STABBR,ZIP,FIPS,OBEREG,CHFNM,CHFTITLE,GENTELE,EIN,UEIS,OPEID,OPEFLAG,WEBADDR,ADMINURL,FAIDURL,APPLURL,NPRICURL,VETURL,ATHURL,DISAURL,SECTOR,ICLEVEL,CONTROL,HLOFFER,UGOFFER,GROFFER,HDEGOFR1,DEGGRANT,HBCU,HOSPITAL,MEDICAL,TRIBAL,LOCALE,OPENPUBL,ACT,NEWID,DEATHYR,CLOSEDAT,CYACTIVE,POSTSEC,PSEFLAG,PSET4FLG,RPTMTH,INSTCAT,C21BASIC,C21IPUG,C21IPGRD,C21UGPRF,C21ENPRF,C21SZSET,C18BASIC,C15BASIC,CCBASIC,CARNEGIE,LANDGRNT,INSTSIZE,F1SYSTYP,F1SYSNAM,F1SYSCOD,CBSA,CBSATYPE,CSA,COUNTYCD,COUNTYNM,CNGDSTCD,LONGITUD,LATITUDE,DFRCGID,DFRCUSCG,domain
0,100654,Alabama A & M University,AAMU,4900 Meridian Street,Normal,AL,35762,1,5,Dr. Daniel K. Wims,President,2563725000,636001109,JDVGS67MSLH7,100200,1,www.aamu.edu/,https://www.aamu.edu/admissions-aid/index.html,https://www.aamu.edu/admissions-aid/financial-...,https://www.aamu.edu/admissions-aid/undergradu...,www.aamu.edu/admissions-aid/tuition-fees/net-p...,,,https://www.aamu.edu/administrativeoffices/VAD...,1,1,1,9,1,1,12,1,1,2,2,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,18,16,18,10,4,14,18,18,18,16,1,3,2,-2,-2,26620,1,290,1089,Madison County,105,-86.568502,34.783368,106,1,
1,100663,University of Alabama at Birmingham,UAB,Administration Bldg Suite 1070,Birmingham,AL,35294-0110,1,5,Ray L. Watts,President,2059344011,636005396,YND4PLMC9AN7,105200,1,https://www.uab.edu/,https://www.uab.edu/admissions/,https://www.uab.edu/cost-aid/,https://www.uab.edu/admissions/apply,https://tcc.ruffalonl.com/University of Alabam...,https://www.uab.edu/students/veterans,https://www.uab.edu/registrar/students,https://www.uab.edu/students/disability/,1,1,1,9,1,1,11,1,2,1,1,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,15,14,14,9,5,15,15,15,15,15,2,5,1,The University of Alabama System,101050,13820,1,142,1073,Jefferson County,107,-86.799345,33.505697,91,1,www.uab.edu
2,100690,Amridge University,Southern Christian University Regions University,1200 Taylor Rd,Montgomery,AL,36117-3553,1,5,Michael C.Turner,President,33438738777550,237034324,RB27R4GLDKE7,2503400,1,https://www.amridgeuniversity.edu/,https://www.amridgeuniversity.edu/admissions/,https://www.amridgeuniversity.edu/financialaid/,https://www.amridgeuniversity.edu/myamridge/,https://www2.amridgeuniversity.edu:9091/,https://www.amridgeuniversity.edu/admissions/m...,,https://www.amridgeuniversity.edu/studentservi...,2,1,2,9,1,1,12,1,2,2,2,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,20,20,18,5,6,6,20,20,21,51,2,1,2,-2,-2,33860,1,388,1101,Montgomery County,102,-86.174010,32.362609,124,2,www.amridgeuniversity.edu
3,100706,University of Alabama in Huntsville,UAH University of Alabama Huntsville,301 Sparkman Dr,Huntsville,AL,35899,1,5,Chuck Karr,President,2568246120,630520830,HB6KNGVNJRU1,105500,1,www.uah.edu/,https://www.uah.edu/admissions,finaid.uah.edu/,register.uah.edu/,finaid.uah.edu/,www.uah.edu/admissions/graduate/financial-aid/...,www.uah.edu/heoa,www.uah.edu/health-and-wellness/disability-sup...,1,1,1,9,1,1,11,1,2,2,2,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,15,17,17,15,4,13,16,16,15,16,2,3,1,The University of Alabama System,101050,26620,1,290,1089,Madison County,105,-86.640449,34.724557,91,1,
4,100724,Alabama State University,,915 S Jackson Street,Montgomery,AL,36104-0271,1,5,Quinton T. Ross,President,3342294100,636001101,DLJWLMSNK627,100500,1,www.alasu.edu/,www.alasu.edu/admissions/index.aspx,www.alasu.edu/undergraduate/expensesandfinanci...,www.alasu.edu/admissions/undergrad-admissions/...,www.alasu.edu/cost-aid/tuition-costs/net-price...,,https://www.alasu.edu/administration/consumer-...,www.alasu.edu/academics/researchcenters/alabam...,1,1,1,9,1,1,11,1,1,2,2,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,17,13,18,10,3,14,19,19,18,21,2,2,2,-2,-2,33860,1,388,1101,Montgomery County,107,-86.295677,32.364317,97,1,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [14]:
services = pd.read_csv('data/ic2023.csv', encoding='latin1')
services

,UNITID,PEO1ISTR,PEO2ISTR,PEO3ISTR,PEO4ISTR,PEO5ISTR,PEO6ISTR,PEO7ISTR,CNTLAFFI,PUBPRIME,PUBSECON,RELAFFIL,LEVEL1,LEVEL1A,LEVEL1B,LEVEL2,LEVEL3,LEVEL4,LEVEL5,LEVEL6,LEVEL7,LEVEL8,LEVEL12,LEVEL17,LEVEL18,LEVEL19,CALSYS,FT_UG,FT_FTUG,FTGDNIDP,PT_UG,PT_FTUG,PTGDNIDP,DOCPP,DOCPPSP,OPENADMP,NONCRDT1,NONCRDT2,NONCRDT3,NONCRDT4,NONCRDT5,NONCRDT6,NONCRDT7,NONCRDT8,NONCRDT9,ENRHSST,ENRHSST1,ENRHSST2,VET1,VET2,VET3,VET4,VET5,VET9,CREDITS2,CREDITS3,CREDITS4,SLO5,SLO51,SLO52,SLO521,SLO53,SLO6,SLO7,SLO8,SLO81,SLO82,SLO83,SLO9,SLOA,SLOB,YRSCOLL,STUSRV2,STUSRV3,STUSRV4,STUSRV8,STUSRV9,LIBRES1,LIBRES2,LIBRES3,LIBRES4,LIBRES5,LIBRES6,LIBRES9,TUITPL,TUITPL1,TUITPL2,TUITPL3,TUITPL4,PRMPGM,DSTNUGC,DSTNUGP,DSTNUGN,DSTNGC,DSTNGP,DSTNGN,DISTCRS,DISTPGS,DSTNCED1,DSTNCED2,DSTNCED3,DISTNCED,DISAB,XDISABPC,DISABPCT,ALLONCAM,TUITVARY,ROOM,XROOMCAP,ROOMCAP,BOARD,XMEALSWK,MEALSWK,XROOMAMT,ROOMAMT,XBORDAMT,BOARDAMT,XRMBDAMT,RMBRDAMT,XAPPFEEU,APPLFEEU,XAPPFEEG,APPLFEEG,ATHASSOC,ASSOC1,ASSOC2,ASSOC3,ASSOC4,ASSOC5,ASSOC6,SPORT1,CONFNO1,SPORT2,CONFNO2,SPORT3,CONFNO3,SPORT4,CONFNO4
0,100654,0,1,0,0,0,0,0,1,2,0,-2,0,0,0,0,0,0,1,0,1,1,0,1,0,0,1,1,1,1,1,1,1,-2,-2,2,0,0,1,0,0,0,0,0,0,1,1,0,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,1,1,1,0,1,0,1,0,-2,1,1,1,1,0,1,1,1,1,1,1,0,1,0,0,1,0,2,1,1,0,1,1,0,1,1,1,1,0,2,1,A,.,2,1,1,R,3620,1,R,21,R,3790,R,4202,A,.,R,30,R,45,1,1,0,0,0,0,0,1,133,1,133,1,133,1,133
1,100663,0,1,1,0,0,0,0,1,2,0,-2,1,0,1,1,0,0,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,2,0,0,1,0,0,0,1,1,0,1,1,0,1,1,1,1,1,0,1,1,0,1,1,0,0,1,1,0,1,1,0,1,0,1,0,-2,1,1,1,0,0,1,1,1,1,1,1,0,1,0,0,1,0,1,1,1,0,1,1,0,1,1,1,1,0,2,2,R,6.94,2,1,1,R,3480,2,A,.,R,8550,R,6030,A,.,R,40,R,50,1,1,0,0,0,0,0,1,372,1,372,1,372,1,372
2,100690,0,1,0,0,0,0,0,4,-2,-2,74,0,0,0,0,1,0,1,0,1,0,0,1,0,0,1,1,2,1,1,1,1,-2,-2,1,0,0,0,0,0,0,0,0,1,1,1,0,1,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-2,1,1,0,0,0,1,1,1,1,1,1,0,2,0,0,0,0,2,1,1,0,1,1,0,1,1,1,1,0,1,1,A,.,-2,2,2,A,.,3,A,.,A,.,A,.,A,.,R,50,R,50,2,0,0,0,0,0,0,2,-2,2,-2,2,-2,2,-2
3,100706,0,1,1,1,0,0,0,1,2,0,-2,1,0,1,0,0,0,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,2,2,1,1,0,1,0,0,0,1,0,1,1,0,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,1,1,1,0,1,0,0,0,-2,1,1,1,1,0,1,1,1,1,1,1,0,1,0,1,1,0,2,1,1,0,1,1,0,1,1,1,1,0,2,2,R,4.80,2,1,1,R,2148,1,R,21,A,.,A,.,R,11122,R,30,R,60,1,1,0,0,0,0,0,2,-2,1,146,1,146,1,146
4,100724,1,1,0,0,0,0,1,1,2,0,-2,0,0,0,0,0,0,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,2,2,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,1,0,0,1,1,0,1,0,0,0,1,1,0,1,0,0,1,0,0,0,1,1,1,1,0,0,1,1,1,1,1,1,0,1,0,0,1,0,1,1,1,0,1,1,0,1,1,1,1,0,2,1,A,.,2,1,1,R,2079,1,R,19,R,3500,R,4190,A,.,R,30,R,30,1,1,0,0,0,0,0,1,133,1,133,1,133,1,133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6044,499662,1,0,-2,0,0,0,0,2,-2,-2,-2,0,0,0,0,0,1,-2,-2,-2,-2,0,-2,-2,-2,1,1,1,-2,1,2,-2,-2,-2,1,0,0,0,0,0,0,0,0,1,2,-2,-2,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,1,-2,-2,-2,-2,0,0,0,-2,1,1,1,0,0,1,1,1,0,1,0,0,1,0,0,1,0,2,0,0,1,-2,-2,-2,0,0,0,-2,1,2,1,A,.,2,2,2,A,.,3,A,.,A,.,A,.,A,.,R,25,A,.,-2,-2,-2,-2,-2,-2,-2,2,-2,2,-2,2,-2,2,-2
6045,499671,0,1,0,0,0,0,0,2,-2,-2,-2,0,0,0,0,1,0,1,0,0,0,0,0,0,0,6,1,1,2,1,1,2,-2,-2,1,0,0,0,0,0,0,0,0,1,2,-2,-2,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-2,1,1,1,0,0,0,1,1,1,1,1,0,1,0,0,1,0,2,1,0,0,-2,-2,-2,1,0,1,-2,0,2,1,A,.,2,-2,2,A,.,3,A,.,A,.,A,.,A,.,R,0,A,.,2,0,0,0,0,0,0,2,-2,2,-2,2,-2,2,-2
6046,499680,1,0,-2,0,0,0,0,2,-2,-2,-2,1,0,1,0,1,0,-2,-2,-2,-2,0,-2,-2,-2,7,1,1,-2,2,2,-2,-2,-2,1,0,0,0,0,0,0,0,0,1,2,-2,-2,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-2,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0,2,1,0,0,-2,-2,-2,1,0,1,-2,0,2

In [15]:
charges = pd.read_csv('data/ic2023_ay.csv', encoding='latin1')
charges

,UNITID,XTUIT1,TUITION1,XFEE1,FEE1,XHRCHG1,HRCHG1,XTUIT2,TUITION2,XFEE2,FEE2,XHRCHG2,HRCHG2,XTUIT3,TUITION3,XFEE3,FEE3,XHRCHG3,HRCHG3,XTUIT5,TUITION5,XFEE5,FEE5,XHRCHG5,HRCHG5,XTUIT6,TUITION6,XFEE6,FEE6,XHRCHG6,HRCHG6,XTUIT7,TUITION7,XFEE7,FEE7,XHRCHG7,HRCHG7,XISPRO1,ISPROF1,XISPFE1,ISPFEE1,XOSPRO1,OSPROF1,XOSPFE1,OSPFEE1,XISPRO2,ISPROF2,XISPFE2,ISPFEE2,XOSPRO2,OSPROF2,XOSPFE2,OSPFEE2,XISPRO3,ISPROF3,XISPFE3,ISPFEE3,XOSPRO3,OSPROF3,XOSPFE3,OSPFEE3,XISPRO4,ISPROF4,XISPFE4,ISPFEE4,XOSPRO4,OSPROF4,XOSPFE4,OSPFEE4,XISPRO5,ISPROF5,XISPFE5,ISPFEE5,XOSPRO5,OSPROF5,XOSPFE5,OSPFEE5,XISPRO6,ISPROF6,XISPFE6,ISPFEE6,XOSPRO6,OSPROF6,XOSPFE6,OSPFEE6,XISPRO7,ISPROF7,XISPFE7,ISPFEE7,XOSPRO7,OSPROF7,XOSPFE7,OSPFEE7,XISPRO8,ISPROF8,XISPFE8,ISPFEE8,XOSPRO8,OSPROF8,XOSPFE8,OSPFEE8,XISPRO9,ISPROF9,XISPFE9,ISPFEE9,XOSPRO9,OSPROF9,XOSPFE9,OSPFEE9,XCHG1AT0,CHG1AT0,XCHG1AF0,CHG1AF0,XCHG1AY0,CHG1AY0,XCHG1AT1,CHG1AT1,XCHG1AF1,CHG1AF1,XCHG1AY1,CHG1AY1,XCHG1AT2,CHG1AT2,XCHG1AF2,CHG1AF2,XCHG1AY2,CHG1AY2,XCHG1AT3,CHG1AT3,XCHG1AF3,CHG1AF3,XCHG1AY3,CHG1AY3,CHG1TGTD,CHG1FGTD,XCHG2AT0,CHG2AT0,XCHG2AF0,CHG2AF0,XCHG2AY0,CHG2AY0,XCHG2AT1,CHG2AT1,XCHG2AF1,CHG2AF1,XCHG2AY1,CHG2AY1,XCHG2AT2,CHG2AT2,XCHG2AF2,CHG2AF2,XCHG2AY2,CHG2AY2,XCHG2AT3,CHG2AT3,XCHG2AF3,CHG2AF3,XCHG2AY3,CHG2AY3,CHG2TGTD,CHG2FGTD,XCHG3AT0,CHG3AT0,XCHG3AF0,CHG3AF0,XCHG3AY0,CHG3AY0,XCHG3AT1,CHG3AT1,XCHG3AF1,CHG3AF1,XCHG3AY1,CHG3AY1,XCHG3AT2,CHG3AT2,XCHG3AF2,CHG3AF2,XCHG3AY2,CHG3AY2,XCHG3AT3,CHG3AT3,XCHG3AF3,CHG3AF3,XCHG3AY3,CHG3AY3,CHG3TGTD,CHG3FGTD,XCHG4AY0,CHG4AY0,XCHG4AY1,CHG4AY1,XCHG4AY2,CHG4AY2,XCHG4AY3,CHG4AY3,XCHG5AY0,CHG5AY0,XCHG5AY1,CHG5AY1,XCHG5AY2,CHG5AY2,XCHG5AY3,CHG5AY3,XCHG6AY0,CHG6AY0,XCHG6AY1,CHG6AY1,XCHG6AY2,CHG6AY2,XCHG6AY3,CHG6AY3,XCHG7AY0,CHG7AY0,XCHG7AY1,CHG7AY1,XCHG7AY2,CHG7AY2,XCHG7AY3,CHG7AY3,XCHG8AY0,CHG8AY0,XCHG8AY1,CHG8AY1,XCHG8AY2,CHG8AY2,XCHG8AY3,CHG8AY3,XCHG9AY0,CHG9AY0,XCHG9AY1,CHG9AY1,XCHG9AY2,CHG9AY2,XCHG9AY3,CHG9AY3
0,100654,R,8610,R,1414,R,287,R,8610,R,1414,R,287,R,17220,R,1414,R,574,R,10128,R,1414,R,422,R,10128,R,1414,R,422,R,20160,R,1414,R,840,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,R,8610,R,1414,R,10024,R,8610,R,1414,R,10024,R,8610,R,1414,R,10024,R,8610,R,1414,R,10024,.,.,R,8610,R,1414,R,10024,R,8610,R,1414,R,10024,R,8610,R,1414,R,10024,R,8610,R,1414,R,10024,.,.,R,17220,R,1414,R,18634,R,17220,R,1414,R,18634,R,17220,R,1414,R,18634,R,17220,R,1414,R,18634,.,.,R,1600,R,1600,R,1600,R,2192,R,9240,R,9520,R,9520,R,11402,R,3090,R,3090,R,3090,R,3864,R,9240,R,9520,R,9520,R,11402,R,3090,R,3090,R,3090,R,3864,R,3440,R,3440,R,3440,R,4271
1,100663,R,8832,R,0,R,368,R,8832,R,0,R,368,R,21864,R,0,R,911,R,8424,R,0,R,468,R,8424,R,0,R,468,R,20556,R,0,R,1142,Y,.,Y,.,Y,.,Y,.,R,31902,R,0,R,74362,R,0,R,32134,R,0,R,62714,R,0,R,29841,R,0,R,54012,R,0,Y,.,Y,.,Y,.,Y,.,Y,.,Y,.,Y,.,Y,.,Y,.,Y,.,Y,.,Y,.,Y,.,Y,.,Y,.,Y,.,Y,.,Y,.,Y,.,Y,.,R,8568,R,0,R,8568,R,8568,R,0,R,8568,R,8832,R,0,R,8832,R,8832,R,0,R,8832,.,.,R,8568,R,0,R,8568,R,8568,R,0,R,8568,R,8832,R,0,R,8832,R,8832,R,0,R,8832,.,.,R,20400,R,0,R,20400,R,20400,R,0,R,20400,R,21216,R,0,R,21216,R,21864,R,0,R,21864,.,.,R,1200,R,1200,R,1200,R,1200,R,12307,R,12307,R,13120,R,13590,R,5555,R,5555,R,5757,R,5757,R,12307,R,12307,R,14330,R,14580,R,5555,R,5555,R,6007,R,6007,R,5555,R,5555,R,6007,R,6007
2,100690,R,9480,R,990,R,455,R,9480,R,990,R,455,R,9480,R,990,R,455,R,11700,R,1390,R,750,R,11700,R,1390,R,750,R,11700,R,1390,R,750,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,.,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,.,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,.,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.,A,.
3,100706,R,10120,R,1758,R,421,R,10120,R,1758,R,421,R,23012,R,1758,R,958,R,10632,R,1054,R,708,R,10632,R,1054,R,708,R,24430,R,1054,R,1628,A,.,A,.,

In [ ]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic, needs_local_scope)
from IPython.core.magic_arguments import (argument, magic_arguments, parse_argstring)
import duckdb

@magic_arguments()
@argument('query', nargs='*', help="The SQL query to execute")
@argument('--db', '-d', default=None, help="Which connection to use")
@argument('--output', '-o', default=None, help="Name of the variable to store output DataFrame in local scope")
@register_line_cell_magic
@needs_local_scope
def quack(line, cell=None, local_ns=None):
    "Magic that works both as %lcmagic and as %%lcmagic"

    args = parse_argstring(quack, line)
    query = cell if cell else ' '.join(args.query)

    if args.db:
        conn = local_ns[args.db]
    elif 'conn' in local_ns:
        conn = local_ns['conn']
    else:
        conn = duckdb.connect(':memory:')
        local_ns['conn'] = conn

    result_df = conn.execute(query).fetchdf()

    if args.output:
        local_ns[args.output] = result_df
    return result_df

# Usage example:
# %%quack -o df SELECT * FROM my_table

In [17]:
%%quack --output peds_df
SELECT
    schools.unitid as college_id,
    trim(INSTNM) AS college_name,
    trim(IALIAS) AS short_name,
    city AS city,
    stabbr AS state,
    LATITUDE::float as latitude,
    LONGITUD::float as longitude,
    (try_cast(TUITION3 AS DECIMAL(10,2)) + try_cast(FEE3 as decimal(10,2)))::bigint AS total_cost,
    domain
FROM schools
JOIN charges ON schools.UNITID = charges.UNITID
JOIN services ON schools.UNITID = services.UNITID

,college_id,college_name,short_name,city,state,latitude,longitude,total_cost,domain
0,100654,Alabama A & M University,AAMU,Normal,AL,34.783367,-86.568504,18634.0,
1,100663,University of Alabama at Birmingham,UAB,Birmingham,AL,33.505695,-86.799347,21864.0,www.uab.edu
2,100706,University of Alabama in Huntsville,UAH University of Alabama Huntsville,Huntsville,AL,34.724556,-86.640450,24770.0,
3,100751,The University of Alabama,,Tuscaloosa,AL,33.211876,-87.545975,33200.0,
4,100830,Auburn University at Montgomery,AUM||Auburn University at Montgomery|Auburn Mo...,Montgomery,AL,32.367359,-86.177544,20092.0,
...,...,...,...,...,...,...,...,...,...
3820,452948,Galen College of Nursing-Cincinnati,,Cincinnati,OH,39.283691,-84.365944,16400.0,
3821,488846,University of the People,,Pasadena,CA,34.146168,-118.138046,1200.0,
3822,499538,Puerto Rico Advance Institute,,Bayamon,PR,18.399160,-66.154488,7395.0,
3823,499635,EDP University of Puerto Rico-Caguas,,Caguas,PR,18.242649,-66.025307,14125.0,www.edpuniversity.edu


In [18]:
import plotly.express as px

# Calculate the average total cost per state
average_cost_per_state = peds_df.groupby('state')['total_cost'].mean().reset_index()

# Create the heat map
fig = px.choropleth(
    average_cost_per_state,
    locations='state',
    locationmode='USA-states',
    color='total_cost',
    color_continuous_scale='Viridis',
    scope='usa',
    labels={'total_cost': 'Mean Avg. Total Cost'},
    title='Average Out of State Cost'
)

fig.show()

In [19]:
import feed
import json
from importlib import reload; reload(feed)

data_file = os.path.join('data', 'utr-mens.json')
if not os.path.exists(data_file):
    print('Fetching data...')
    colleges = await feed.utr_colleges(top=10000)
    with open(data_file, 'w') as f:
        json.dump(colleges, f, indent=2)
print(f'Loading data from {data_file}...')
json.loads(open(data_file).read())
print(f"Number of colleges: {len(colleges['hits'])}")

Loading data from data/utr-mens.json...


NameError: name 'colleges' is not defined

In [ ]:
import pandas as pd
import json

# Load the JSON data from the file
with open(data_file, 'r') as f:
    data = json.load(f)

# Extract the 'hits' property which contains the rows
hits = data['hits']

# Normalize the JSON data to flatten the attributes into new columns
utr = pd.json_normalize(hits)

utr['domain'] = utr['source.url'].apply(parse_domain)
# Display the dataframe
utr.head()

,fields,index,score,type,version,id,nested,parent,routing,sorts,explanation,matchedQueries,source.clubMemberRole,source.id,source.name,source.description,source.locationId,source.tierTypeId,source.clubSubTypeId,source.paymentsEnabled,source.sanctioned,source.url,source.memberCount,source.eventCount,source.private,source.isTrialing,source.trialEndDateUtc,source.isSearchable,source.canRunEvents,source.isHighSchool,source.isCollege,source.gender,source.bannerUrl,source.profilePhotoUrl,source.location.locationId,source.location.latLng,source.location.display,source.location.googleFormattedName,source.location.googlePlaceId,source.location.placeName,source.location.streetAddress,source.location.cityStateZip,source.location.cityName,source.location.cityAbbr,source.location.stateName,source.location.stateAbbr,source.location.countryName,source.location.countryCode2,source.location.countryCode3,source.location.countryIoc,source.statusType,source.school.displayName,source.school.altName1,source.school.altName2,source.school.altName3,source.school.altName4,source.school.nickname,source.school.altNickname1,source.school.altNickname2,source.school.ladyNickname,source.school.private,source.school.type,source.school.power6Avg,source.school.conference.id,source.school.conference.conferenceName,source.school.conference.shortName,source.school.conference.divisionId,source.school.conference.division.id,source.school.conference.division.divisionName,source.school.conference.division.shortName,source.school.rosterHasUnclaimedPlayers,source.school.name,source.school.clubId,source.school.shortName,source.school.profilePhotoUrl,source.school.rosterCount,source.school.rosterYear,source.school.power6,source.school.power6High,source.school.power6Low,source.school.power6Men,source.school.power6MenHigh,source.school.power6MenLow,source.school.power6Women,source.school.power6WomenHigh,source.school.power6WomenLow,source.school.isCollege,source.school.tierType,source.school.clubSubTypeId,source.school.roster,source.school.clubProfileImages.default,source.school.clubProfileImages.thumbnail.oneX,source.school.clubProfileImages.thumbnail.twoX,source.school.clubProfileImages.thumbnail.threeX,source.school.clubProfileImages.card.oneX,source.school.clubProfileImages.card.twoX,source.school.clubProfileImages.card.threeX,source.school.clubProfileImages.profile.oneX,source.school.clubProfileImages.profile.twoX,source.school.clubProfileImages.profile.threeX,source.school.clubProfileImages.icon.oneX,source.school.clubProfileImages.icon.twoX,source.school.clubProfileImages.icon.threeX,source.school.currentMember,source.clubMembersSimplified,source.clubImages.default,source.clubImages.banner.oneX,source.clubImages.banner.twoX,source.clubImages.banner.threeX,source.clubImages.dashboard.oneX,source.clubImages.dashboard.twoX,source.clubImages.dashboard.threeX,source.clubProfileImages.default,source.clubProfileImages.thumbnail.oneX,source.clubProfileImages.thumbnail.twoX,source.clubProfileImages.thumbnail.threeX,source.clubProfileImages.card.oneX,source.clubProfileImages.card.twoX,source.clubProfileImages.card.threeX,source.clubProfileImages.profile.oneX,source.clubProfileImages.profile.twoX,source.clubProfileImages.profile.threeX,source.clubProfileImages.icon.oneX,source.clubProfileImages.icon.twoX,source.clubProfileImages.icon.threeX,source.clubTags,source.school.conference,domain
0,None,clubs_v2710,None,default,None,929,None,None,None,[82.06],None,[],None,929,University of Texas (Austin) - M,None,130,6,1,False,True,http://www.texassports.com/index.aspx?path=mten,359,6,False,False,None,False,True,False,True,Male,None,929/images/profile/db64fe26-b41e-4793-97e0-5e9...,None,"[30.267153, -97.7430608]","Austin, TX","Austin, TX, USA",None,None,,"Austin, TX",Austin,Austin,Texas,TX,United States,US,USA,USA,None,University of Texas (Austin),None,None,None,None,None,None,None,None,False,College,None,26.0,Southeastern Conference,SEC,NaN,1.0,NCAA - Division I,D1,True,University of T

In [ ]:
%%quack --output matches

SELECT
    (
        SELECT
            array_agg(peds_df.college_id)
        FROM peds_df
        WHERE regexp_replace(peds_df.college_name, '\W', '', 'g') IN (
            regexp_replace("source.school.name", '\W', '', 'g'),
            regexp_replace("source.school.displayName", '\W', '', 'g'),
            regexp_replace("source.school.altName1", '\W', '', 'g'),
            regexp_replace("source.school.altName2", '\W', '', 'g')
        )
    ) AS peds_college_ids,
    *
FROM utr

,peds_college_ids,fields,index,score,type,version,id,nested,parent,routing,sorts,explanation,matchedQueries,source.clubMemberRole,source.id,source.name,source.description,source.locationId,source.tierTypeId,source.clubSubTypeId,source.paymentsEnabled,source.sanctioned,source.url,source.memberCount,source.eventCount,source.private,source.isTrialing,source.trialEndDateUtc,source.isSearchable,source.canRunEvents,source.isHighSchool,source.isCollege,source.gender,source.bannerUrl,source.profilePhotoUrl,source.location.locationId,source.location.latLng,source.location.display,source.location.googleFormattedName,source.location.googlePlaceId,source.location.placeName,source.location.streetAddress,source.location.cityStateZip,source.location.cityName,source.location.cityAbbr,source.location.stateName,source.location.stateAbbr,source.location.countryName,source.location.countryCode2,source.location.countryCode3,source.location.countryIoc,source.statusType,source.school.displayName,source.school.altName1,source.school.altName2,source.school.altName3,source.school.altName4,source.school.nickname,source.school.altNickname1,source.school.altNickname2,source.school.ladyNickname,source.school.private,source.school.type,source.school.power6Avg,source.school.conference.id,source.school.conference.conferenceName,source.school.conference.shortName,source.school.conference.divisionId,source.school.conference.division.id,source.school.conference.division.divisionName,source.school.conference.division.shortName,source.school.rosterHasUnclaimedPlayers,source.school.name,source.school.clubId,source.school.shortName,source.school.profilePhotoUrl,source.school.rosterCount,source.school.rosterYear,source.school.power6,source.school.power6High,source.school.power6Low,source.school.power6Men,source.school.power6MenHigh,source.school.power6MenLow,source.school.power6Women,source.school.power6WomenHigh,source.school.power6WomenLow,source.school.isCollege,source.school.tierType,source.school.clubSubTypeId,source.school.roster,source.school.clubProfileImages.default,source.school.clubProfileImages.thumbnail.oneX,source.school.clubProfileImages.thumbnail.twoX,source.school.clubProfileImages.thumbnail.threeX,source.school.clubProfileImages.card.oneX,source.school.clubProfileImages.card.twoX,source.school.clubProfileImages.card.threeX,source.school.clubProfileImages.profile.oneX,source.school.clubProfileImages.profile.twoX,source.school.clubProfileImages.profile.threeX,source.school.clubProfileImages.icon.oneX,source.school.clubProfileImages.icon.twoX,source.school.clubProfileImages.icon.threeX,source.school.currentMember,source.clubMembersSimplified,source.clubImages.default,source.clubImages.banner.oneX,source.clubImages.banner.twoX,source.clubImages.banner.threeX,source.clubImages.dashboard.oneX,source.clubImages.dashboard.twoX,source.clubImages.dashboard.threeX,source.clubProfileImages.default,source.clubProfileImages.thumbnail.oneX,source.clubProfileImages.thumbnail.twoX,source.clubProfileImages.thumbnail.threeX,source.clubProfileImages.card.oneX,source.clubProfileImages.card.twoX,source.clubProfileImages.card.threeX,source.clubProfileImages.profile.oneX,source.clubProfileImages.profile.twoX,source.clubProfileImages.profile.threeX,source.clubProfileImages.icon.oneX,source.clubProfileImages.icon.twoX,source.clubProfileImages.icon.threeX,source.clubTags,source.school.conference,domain
0,[176080],NaN,clubs_v2710,NaN,default,NaN,1268,NaN,NaN,NaN,[79.7],NaN,[],NaN,1268,Mississippi State University - M,None,3361,6,1,False,True,None,56,25,False,False,NaN,False,True,False,True,Male,1268/images/banner/47add3d8-d8dc-4035-beb2-a7f...,1268/images/profile/e382d392-6515-4fdd-ad25-d7...,NaN,"[33.4503998, -88.81838719999999]","Starkville, MS","Starkville, MS 39759, USA",NaN,None,,"Starkville, MS 39759",Starkville,Starkville,Mississippi,MS,United States,US,USA,USA,NaN,Mississippi State University,Miss St,MSU,None,None,Bulldogs,None,None,None,False,College,NaN,26.0,Southeas

In [ ]:
%%quack

-- output matched

SELECT
    (
        SELECT
            array_agg(peds_df.college_id)
        FROM peds_df
        WHERE (utr.domain is not null AND peds_df.domain is not null AND utr.domain = peds_df.domain)
    ) AS peds_college_ids,
    * EXCLUDE (peds_college_ids)
FROM matches AS utr
WHERE peds_college_ids IS NULL

,peds_college_ids,fields,index,score,type,version,id,nested,parent,routing,sorts,explanation,matchedQueries,source.clubMemberRole,source.id,source.name,source.description,source.locationId,source.tierTypeId,source.clubSubTypeId,source.paymentsEnabled,source.sanctioned,source.url,source.memberCount,source.eventCount,source.private,source.isTrialing,source.trialEndDateUtc,source.isSearchable,source.canRunEvents,source.isHighSchool,source.isCollege,source.gender,source.bannerUrl,source.profilePhotoUrl,source.location.locationId,source.location.latLng,source.location.display,source.location.googleFormattedName,source.location.googlePlaceId,source.location.placeName,source.location.streetAddress,source.location.cityStateZip,source.location.cityName,source.location.cityAbbr,source.location.stateName,source.location.stateAbbr,source.location.countryName,source.location.countryCode2,source.location.countryCode3,source.location.countryIoc,source.statusType,source.school.displayName,source.school.altName1,source.school.altName2,source.school.altName3,source.school.altName4,source.school.nickname,source.school.altNickname1,source.school.altNickname2,source.school.ladyNickname,source.school.private,source.school.type,source.school.power6Avg,source.school.conference.id,source.school.conference.conferenceName,source.school.conference.shortName,source.school.conference.divisionId,source.school.conference.division.id,source.school.conference.division.divisionName,source.school.conference.division.shortName,source.school.rosterHasUnclaimedPlayers,source.school.name,source.school.clubId,source.school.shortName,source.school.profilePhotoUrl,source.school.rosterCount,source.school.rosterYear,source.school.power6,source.school.power6High,source.school.power6Low,source.school.power6Men,source.school.power6MenHigh,source.school.power6MenLow,source.school.power6Women,source.school.power6WomenHigh,source.school.power6WomenLow,source.school.isCollege,source.school.tierType,source.school.clubSubTypeId,source.school.roster,source.school.clubProfileImages.default,source.school.clubProfileImages.thumbnail.oneX,source.school.clubProfileImages.thumbnail.twoX,source.school.clubProfileImages.thumbnail.threeX,source.school.clubProfileImages.card.oneX,source.school.clubProfileImages.card.twoX,source.school.clubProfileImages.card.threeX,source.school.clubProfileImages.profile.oneX,source.school.clubProfileImages.profile.twoX,source.school.clubProfileImages.profile.threeX,source.school.clubProfileImages.icon.oneX,source.school.clubProfileImages.icon.twoX,source.school.clubProfileImages.icon.threeX,source.school.currentMember,source.clubMembersSimplified,source.clubImages.default,source.clubImages.banner.oneX,source.clubImages.banner.twoX,source.clubImages.banner.threeX,source.clubImages.dashboard.oneX,source.clubImages.dashboard.twoX,source.clubImages.dashboard.threeX,source.clubProfileImages.default,source.clubProfileImages.thumbnail.oneX,source.clubProfileImages.thumbnail.twoX,source.clubProfileImages.thumbnail.threeX,source.clubProfileImages.card.oneX,source.clubProfileImages.card.twoX,source.clubProfileImages.card.threeX,source.clubProfileImages.profile.oneX,source.clubProfileImages.profile.twoX,source.clubProfileImages.profile.threeX,source.clubProfileImages.icon.oneX,source.clubProfileImages.icon.twoX,source.clubProfileImages.icon.threeX,source.clubTags,source.school.conference,domain
0,"[100654, 100706, 100751, 100830, 100858, 10114...",NaN,clubs_v2710,NaN,default,NaN,1420,NaN,NaN,NaN,[62.83],NaN,[],NaN,1420,"Union College, Kentucky - M",None,4976,6,1,False,False,www.gounionbulldogs.com,36,16,False,False,NaN,False,True,False,True,Male,1420/images/banner/5ea00392-2a4a-47ca-a690-0fc...,1420/images/profile/ff121eff-2e50-4b1d-bae4-eb...,NaN,"[36.8664765, -83.8888138]","Barbourville, KY","Barbourville, KY, USA",NaN,None,,"Barbourville, KY",Barbourville,Barbourville,Kentucky,KY,United States,US,USA,USA,NaN,"Union College, Kentucky",None,None,None,None,None,None,N

In [ ]:
%%quack --output joined_df

SELECT
    "source.id" as utr_college_id,
--    peds_df.college_name,
--    peds_df.short_name,
    "source.school.name" as utr_college_name,
    "source.school.displayName" as utr_college_display_name,
    "source.school.power6"     as power6,
    "source.school.power6High" as power6High,
    "source.school.power6Low"  as power6Low,
    array_agg(peds_df.college_id)   as peds_college_ids,
    array_agg(peds_df.college_name) as peds_college_names
FROM utr
JOIN peds_df ON  state = "source.location.stateAbbr"
    AND (
        peds_df.college_name IN ("source.school.name", "source.school.displayName", "source.school.altName1", "source.school.altName2")
        OR (utr.domain is not null AND peds_df.domain is not null AND utr.domain = peds_df.domain)
        OR (peds_df.short_name is not null AND "source.school.shortName" is not null AND peds_df.short_name = "source.school.shortName")
        -- OR (ST_Distance(ST_Point(peds_df.LATITUDE, peds_df.longitude), ST_Point("source.location.latLng"[1], "source.location.latLng"[2])) < 0.05)
    )
WHERE peds_df.college_id IS NOT NULL
  -- AND "source.id" = 1735
GROUP BY 1, 2, 3, 4, 5, 6

,utr_college_id,utr_college_name,utr_college_display_name,power6,power6High,power6Low,peds_college_ids,peds_college_names
0,1520,Delta State University,Delta State University,NaN,NaN,NaN,[175616],[Delta State University]
1,1274,University of Delaware,University of Delaware,NaN,NaN,NaN,[130943],[University of Delaware]
2,2172,Wartburg College,Wartburg College,NaN,NaN,NaN,[154527],[Wartburg College]
3,2607,Friends University,Friends University,NaN,NaN,NaN,[155089],[Friends University]
4,672338,Converse University,Converse College,NaN,NaN,NaN,[217961],[Converse University]
...,...,...,...,...,...,...,...,...
667,1825,Ithaca College,Ithaca College,NaN,NaN,NaN,[191968],[Ithaca College]
668,2193,Hamilton College,Hamilton College,NaN,NaN,NaN,[191515],[Hamilton College]
669,2920,Reedley College,Reedley College,NaN,NaN,NaN,[117052],[Reedley College]
670,1298,Fresno City College,Fresno City College,NaN,NaN,NaN,[114789],[Fresno City College]


In [ ]:
%%quack --output joined_df

SELECT
    "source.id" as utr_college_id,
--    peds_df.college_name,
--    peds_df.short_name,
    "source.school.name" as utr_college_name,
    "source.school.displayName" as utr_college_display_name,
    "source.school.power6"     as power6,
    "source.school.power6High" as power6High,
    "source.school.power6Low"  as power6Low,
    array_agg(peds_df.college_id)   as peds_college_ids,
    array_agg(peds_df.college_name) as peds_college_names
FROM utr
JOIN peds_df ON  state = "source.location.stateAbbr"
    AND (
        peds_df.college_name IN ("source.school.name", "source.school.displayName", "source.school.altName1", "source.school.altName2")
        OR (utr.domain is not null AND peds_df.domain is not null AND utr.domain = peds_df.domain)
        OR (peds_df.short_name is not null AND "source.school.shortName" is not null AND peds_df.short_name = "source.school.shortName")
        -- OR (ST_Distance(ST_Point(peds_df.LATITUDE, peds_df.longitude), ST_Point("source.location.latLng"[1], "source.location.latLng"[2])) < 0.05)
    )
WHERE peds_df.college_id IS NOT NULL
  -- AND "source.id" = 1735
GROUP BY 1, 2, 3, 4, 5, 6

,utr_college_id,utr_college_name,utr_college_display_name,power6,power6High,power6Low,peds_college_ids,peds_college_names
0,2712,Community College of Rhode Island,Community College of Rhode Island,NaN,NaN,NaN,[217475],[Community College of Rhode Island]
1,2534,Central College,Central College,NaN,NaN,NaN,[153108],[Central College]
2,1282,William Carey University,William Carey University,NaN,NaN,NaN,[176479],[William Carey University]
3,1604,Delaware State University,Delaware State University,NaN,NaN,NaN,[130934],[Delaware State University]
4,2139,Edgewood College,Edgewood College,NaN,NaN,NaN,[238661],[Edgewood College]
...,...,...,...,...,...,...,...,...
667,1152295,Hudson Valley Community College,Hudson Valley Community College,NaN,NaN,NaN,[191719],[Hudson Valley Community College]
668,1286,Fordham University,Fordham University,NaN,NaN,NaN,[191241],[Fordham University]
669,2910,Fullerton College,Fullerton College,NaN,NaN,NaN,[114859],[Fullerton College]
670,2797,Diablo Valley College,Diablo Valley College,NaN,NaN,NaN,[113634],[Diablo Valley College]


In [ ]:
%%quack

SELECT
    utr_college_name,
    length(peds_college_ids) as count,
FROM joined_df
WHERE length(peds_college_ids) > 1


,utr_college_name,count
0,Queens University of Charlotte,80
1,Doane College,19
2,"Union College, Kentucky",24
3,Indiana University East,36
4,Milligan University,54
5,Lake Forest College,99
6,Concordia University (Illinois),99
7,Georgia Gwinnett College,74
8,College Of Lake County,99
9,Marion Military Institute,40


In [ ]:
%%quack --output tmp

SELECT
    "source.school.name",
    "source.school.displayName"
FROM utr
WHERE "source.school.name" <> "source.school.displayName"
;

,source.school.name,source.school.displayName
0,North Carolina,University of North Carolina
1,UNC Charlotte,Charlotte
2,University of Louisiana at Lafayette,University Of Louisiana at Lafayette
3,Virginia Tech,Virginia Tech
4,University of Texas at San Antonio,University of Texas - San Antonio
5,Weber State,Weber State University
6,Tennessee Wesleyan,Tennessee Wesleyan College
7,Sacramento State,Sacramento State University
8,University of Nebraska Omaha,Omaha
9,Indiana University-Purdue University Indianapolis,IU-Indianapolis


In [ ]:
%%quack --output tmp

SELECT
    *
FROM peds_df
WHERE college_name like '%Louisiana%'
;

,college_id,college_name,short_name,city,state,latitude,longitude,total_cost,domain
0,158088,Central Louisiana Technical Community College,,Alexandria,LA,31.310461,-92.446693,4098.0,
1,159373,Louisiana State University Health Sciences Cen...,LSUHSC LSUHSC-NO LSUMC LSU Medical Center ...,New Orleans,LA,29.957272,-90.083145,18851.0,
2,159382,Louisiana State University-Alexandria,LSUA,Alexandria,LA,31.178616,-92.414406,14725.0,
3,159407,Louisiana State University-Eunice,LSU Eunice,Eunice,LA,30.473291,-92.434525,10166.0,www.lsue.edu
4,159416,Louisiana State University-Shreveport,LSUS,Shreveport,LA,32.428467,-93.705612,20481.0,
5,159568,Louisiana Christian University,,Pineville,LA,31.324528,-92.425972,19750.0,
6,159647,Louisiana Tech University,"LA Tech, Louisiana Tech, Tech",Ruston,LA,32.528297,-92.649963,16866.0,www.latech.edu
7,159993,University of Louisiana at Monroe,ULM; University of Louisiana-Monroe; Uni of LA...,Monroe,LA,32.527748,-92.073738,21290.0,
8,160010,Northwest Louisiana Technical Community College,NLTCC or Northwest LTCC,Minden,LA,32.586037,-93.263023,4109.0,
9,160038,Northwestern State University of Louisiana,NSU | NSULA | Northwestern State University | ...,Natchitoches,LA,31.750666,-93.097794,19652.0,www.nsula.edu
